In [7]:
%matplotlib inline
from costcla.models import CostSensitiveRandomForestClassifier, CostSensitiveDecisionTreeClassifier, CostSensitiveLogisticRegression
from costcla.metrics import cost_loss
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import random

traindf = pd.read_hdf('./german/train_h5.h5', 'df')
testdf = pd.read_hdf('./german/test_h5.h5', 'df')
pd = traindf.append(testdf).reset_index()
m = pd.amount.max()
mc10, mc01 = pd.c10.max(), pd.c01.max()
pd.c10 = [x/mc10 for x in list(pd.c10)]
pd.c01 = [x/mc01 for x in list(pd.c01)]

In [9]:
accuracy = []
misclass_cost = []
toptenacc = []
cost = []

for i in range(0, 10):
    msk = np.random.rand(len(pd)) < 0.8
    traindf = pd[msk].reset_index()
    testdf = pd[~msk].reset_index()

    srt= np.sort(np.asarray(list(testdf['amount'])))
    topten = srt[int(np.ceil(srt.shape[0]*0.90))]
    toptendf = testdf.loc[testdf['amount'] > topten]

    c10train, c10test = list(traindf['c10']), list(testdf['c10'])
    c01train, c01test = list(traindf['c01']), list(testdf['c01'])

    Y_test = list(testdf['label'])
    totaltopten = 0
    correcttopten = 0
    misclass = []
    forest = CostSensitiveDecisionTreeClassifier()
    #forest = CostSensitiveRandomForestClassifier(n_estimators=)
    xtrainlst = list(traindf['x'])
    ind = [x for x in range(0, len(xtrainlst))]
    random.shuffle(ind)

    trainamount = list(traindf['amount'])
    labels = list(traindf['label'])
    Y_train = np.ones((len(xtrainlst)))
    X_train = np.ones((len(xtrainlst), xtrainlst[0].shape[0]))
    train_cost = np.ones((len(xtrainlst), 4))
    for i in range (0, len(xtrainlst)):
        X_train[ind[i], :] = xtrainlst[i]
        Y_train[ind[i]] =  labels[i]
        train_cost[ind[i], :] = [c10train[i], c01train[i], 0, 0]
        
    xtestlst = list(testdf['x'])
    testamount = list(testdf['amount'])
    X_test = np.ones((len(xtestlst), xtestlst[0].shape[0]))
    test_cost = np.ones((len(xtestlst), 4))
    for i in range (0, len(xtestlst)):
        X_test[i, :] = xtestlst[i]
        test_cost[i, :] = [c10test[i], c01test[i], 0, 0]

    forest.fit(X_train, Y_train, train_cost)
    pred = list(forest.predict(X_test))
    misclass = []
    l1 = []
    l2 = []
    for i in range(0, len(pred)):
        if pred[i]!=Y_test[i]:
            if pred[i] == 0:
                misclass.append(test_cost[i, 0])
            else:
                misclass.append(test_cost[i, 1])
            misclass.append(i)
        if testamount[i] > topten:
            totaltopten+=1
            l1.append(Y_test[i])
            l2.append(pred[i])
            if pred[i]==Y_test[i]:
                correcttopten+=1
    toptenacc.append(accuracy_score(l1, l2))
    misclass_cost.append(np.sum(misclass))
    accuracy.append(accuracy_score(Y_test, pred))
    cost.append(cost_loss(Y_test, pred, test_cost))

print(np.mean(accuracy), np.std(accuracy))
print(np.mean(cost), np.std(cost))
print(np.mean(toptenacc), np.std(toptenacc))

0.47424988746655056 0.05803398781578242
3.8869803668187304 0.8891039259798245
0.6613794289645683 0.09577003863527359


In [11]:
print(np.mean(accuracy)*100, np.std(accuracy)*100)
print(np.mean(cost), np.std(cost))
print(np.mean(toptenacc)*100, np.std(toptenacc)*100)

47.42498874665505 5.803398781578243
3.8869803668187304 0.8891039259798245
66.13794289645682 9.57700386352736
